In [4]:
# 3 = Name
# 5 = Roster Position
# 6 = Salary
# 9 = Expected Points
#pg sg sf pf c g uftil
using JuMP, Gurobi,PyPlot, Mosek
result = zeros(19) #[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
cap = collect(5000:5000:100000) #[10000,20000,30000,40000,50000,60000,70000,80000,90000,100000]


for z in 1:19

    playerList = readcsv("DKsalaries2.csv")

    numPlayers = trunc(Int,(length(playerList)/9))
    #m = Model(solver=GurobiSolver(OutputFlag=0))
    m = Model(solver=MosekSolver(LOG=0))
    @variable(m, players[2:numPlayers], Bin)
    @variable(m, pg[2:numPlayers], Bin)
    @variable(m, sg[2:numPlayers], Bin)
    @variable(m, sf[2:numPlayers], Bin)
    @variable(m, pf[2:numPlayers], Bin)
    @variable(m, c[2:numPlayers], Bin)
    @variable(m, f[2:numPlayers], Bin)
    @variable(m, g[2:numPlayers], Bin)
    @variable(m, util[2:numPlayers], Bin)

    @constraint(m, sum(players[i]  for i in 2:numPlayers) <= 8)

        for i in 2:numPlayers
            if contains(playerList[i,5],"PG")
                @constraint(m,pg[i] <= 1)
            else
                @constraint(m,pg[i] == 0)
            end
            if contains(playerList[i,5],"SG")
                @constraint(m,sg[i] <= 1)
            else
               @constraint(m,sg[i] == 0)
            end
            if contains(playerList[i,5],"SF")
                @constraint(m,sf[i] <= 1)
            else
                @constraint(m,sf[i] == 0)
            end
            if contains(playerList[i,5],"PF")
                @constraint(m,pf[i] <= 1)
            else
                @constraint(m,pf[i] == 0)
            end
             if contains(playerList[i,5],"C")
                @constraint(m,c[i] <= 1)
            else
                @constraint(m,c[i] == 0)
            end       
            if contains(playerList[i,5],"G")
                @constraint(m,g[i] <= 1)
            else
                @constraint(m,g[i] == 0)
            end
            if contains(playerList[i,5],"F")
                @constraint(m,f[i] <= 1)
            else
                @constraint(m,f[i] == 0)
            end
            if contains(playerList[i,5],"UTIL")
                @constraint(m,util[i] <= 1)
            else
                @constraint(m,util[i] == 0)
            end

        end  

        @constraint(m, sum(players[i]*playerList[i,6] for i in 2:numPlayers) <= cap[z])

        for i in 2:numPlayers
                @constraint(m, pg[i] + sg[i] + sf[i]+g[i] + pf[i] + c[i] + f[i] + util[i] - players[i] == 0)
        end

        @constraint(m, sum(pg[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(sg[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(sf[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(pf[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(c[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(f[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(g[i] for i in 2:numPlayers) <= 1 )
        @constraint(m, sum(util[i] for i in 2:numPlayers) <= 1 )



        @objective(m, Max, sum(players[i] * playerList[i,9] for i in 2:92))

    #    @constraint(m, players[i])

            solve(m);
           val = getobjectivevalue(m)
            result[z]= trunc(Int,val)

                                                println("Expected Points ", getobjectivevalue(m), " Cash: ",cap[z])
                                                                                
p = getvalue(players)
pg = getvalue(pg)
sg = getvalue(sg)
sf = getvalue(sf)
pf = getvalue(pf)
g = getvalue(g)
f = getvalue(f)
c = getvalue(c)
util = getvalue(util)
           # println(getvalue(f))
for i in 2:numPlayers 
    if trunc(p[i]) > 0 && pg[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " PG")
    end
    if trunc(p[i]) > 0 && sg[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " SG")
    end
    if trunc(p[i]) > 0 && sf[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " SF")
    end
    if trunc(p[i]) > 0 && pf[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " PF")
    end
                                                if trunc(p[i]) > 0 && g[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " G")
    end
    if trunc(p[i]) > 0 && f[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " F")
    end
    if trunc(p[i]) > 0 && c[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " C")
    end
    if trunc(p[i]) > 0 && util[i] > .1
        println(playerList[i,3], " ", playerList[i,6], " ",playerList[i,9], " UTIL")
    end
end
                                            
                                                println()                     
                                            
                                            end

  
    for i in 1:10:100
       # println("Cash to spend: ",cap[i]," Max points: ", result[i])
    end
 
#k=10
#npts = 100
#xfine = linspace(0,10000,npts)
#ffine = ones(npts)
#for j = 1:k
#    ffine = [ffine.*xfine ones(npts)]
#end
#yfine = ffine * result
figure(figsize=(8,4))
plot(cap,result,"r.", markersize=10)
#plot( xfine, yfine, "b-")
axis([0,100000,0,350])
grid("on")



Expected Points 27.1 Cash: 5000
Pau Gasol 4500 27.1 C

Expected Points 60.870000000000005 Cash: 10000
Hassan Whiteside 5400 33.77 UTIL
Pau Gasol 4500 27.1 C

Expected Points 88.37 Cash: 15000
Hassan Whiteside 5400 33.77 UTIL
Kyle Anderson 4000 22.58 F
T.J. McConnell 3000 17.92 G
Jerryd Bayless 2500 14.1 PG

Expected Points 118.40000000000023 Cash: 20000
Hassan Whiteside 5400 33.77 UTIL
Pau Gasol 4500 27.1 C
Tyler Johnson 3900 22.1 G
Bam Adebayo 3200 17.51 F
T.J. McConnell 3000 17.92 PG

Expected Points 145.1900000000004 Cash: 25000
Hassan Whiteside 5400 33.77 UTIL
Pau Gasol 4500 27.1 C
E'Twaun Moore 4100 21.72 F
Kyle Anderson 4000 22.58 SF
Tyler Johnson 3900 22.1 PG
T.J. McConnell 3000 17.92 G

Expected Points 171.53000000000003 Cash: 30000
Hassan Whiteside 5400 33.77 C
Pau Gasol 4500 27.1 UTIL
Kyle Anderson 4000 22.58 SF
Tyler Johnson 3900 22.1 SG
David West 3300 16.45 PF
Bam Adebayo 3200 17.51 F
T.J. McConnell 3000 17.92 G
Jerryd Bayless 2500 14.1 PG

Expected Points 197.12 Cash: 350

LoadError: [91mPyError (ccall(@pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, arg, kw)) <type 'exceptions.ValueError'>
ValueError(u'x and y must have same first dimension, but have shapes (20,) and (19,)',)
  File "/Users/BrendonLapp/Library/Python/2.7/lib/python/site-packages/matplotlib/pyplot.py", line 3317, in plot
    ret = ax.plot(*args, **kwargs)
  File "/Users/BrendonLapp/Library/Python/2.7/lib/python/site-packages/matplotlib/__init__.py", line 1898, in inner
    return func(ax, *args, **kwargs)
  File "/Users/BrendonLapp/Library/Python/2.7/lib/python/site-packages/matplotlib/axes/_axes.py", line 1406, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/Users/BrendonLapp/Library/Python/2.7/lib/python/site-packages/matplotlib/axes/_base.py", line 407, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/Users/BrendonLapp/Library/Python/2.7/lib/python/site-packages/matplotlib/axes/_base.py", line 385, in _plot_args
    x, y = self